# Лабораторная работа №4. Реализация приложения по распознаванию номеров домов.


In [38]:
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

## Задание 1.
Реализуйте глубокую нейронную сеть (полносвязную или сверточную) и обучите ее на синтетических данных (например, наборы MNIST (http://yann.lecun.com/exdb/mnist/) или notMNIST).
Ознакомьтесь с имеющимися работами по данной тематике: англоязычная статья (http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf), видео на YouTube (https://www.youtube.com/watch?v=vGPI_JvLoN0).

In [39]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

network = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, Activation('softmax'))
])

network.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dense_9 (Dense)              (None, 10)               

In [41]:
network.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
network.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=128, epochs=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 27s 447us/sample - loss: 0.8341 - accuracy: 0.9351 - val_loss: 0.0671 - val_accuracy: 0.9768
Epoch 2/10
60000/60000 [==============================] - 26s 440us/sample - loss: 0.0469 - accuracy: 0.9851 - val_loss: 0.0630 - val_accuracy: 0.9813
Epoch 3/10
60000/60000 [==============================] - 25s 425us/sample - loss: 0.0301 - accuracy: 0.9898 - val_loss: 0.0556 - val_accuracy: 0.9821
Epoch 4/10
60000/60000 [==============================] - 27s 456us/sample - loss: 0.0184 - accuracy: 0.9935 - val_loss: 0.0568 - val_accuracy: 0.9854
Epoch 5/10
60000/60000 [==============================] - 25s 414us/sample - loss: 0.0155 - accuracy: 0.9950 - val_loss: 0.0585 - val_accuracy: 0.9850
Epoch 6/10
60000/60000 [==============================] - 26s 428us/sample - loss: 0.0216 - accuracy: 0.9930 - val_loss: 0.0574 - val_accuracy: 0.9866
Epoch 7/10
60000/60000 [====================

In [42]:
network.evaluate(X_test, y_test)

10000/10000 [==============================] - 2s 185us/sample - loss: 0.0570 - accuracy: 0.9872


[0.057024893728429775, 0.9872]

## Задание 2.
После уточнения модели на синтетических данных попробуйте обучить ее на реальных данных (набор Google Street View). Что изменилось в модели?

In [43]:
import os
import urllib


dataset_path = os.path.join('data', 'housenumbers')
os.makedirs(dataset_path, exist_ok=True)

files_for_download = [
    ('http://ufldl.stanford.edu/housenumbers/train_32x32.mat', os.path.join(dataset_path, 'train_32x32.mat')),
    ('http://ufldl.stanford.edu/housenumbers/test_32x32.mat', os.path.join(dataset_path, 'test_32x32.mat')),
]

for url, dest in files_for_download:
    if not os.path.exists(dest):
        urllib.request.urlretrieve(url, dest)

In [44]:
import scipy.io
from sklearn.model_selection import train_test_split

train = scipy.io.loadmat(os.path.join(dataset_path, 'train_32x32.mat'))
X_train, y_train = train["X"], train["y"]

X_train = np.moveaxis(X_train, -1, 0)
y_train = y_train.flatten() - 1

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [45]:
test = scipy.io.loadmat(os.path.join(dataset_path, 'test_32x32.mat'))
X_test, y_test = test["X"], test["y"]

X_test = np.moveaxis(X_test, -1, 0)
y_test = y_test.flatten() - 1

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

network = Sequential([
    Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, Activation('softmax'))
])

network.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               2097664   
_________________________________________________________________
dense_11 (Dense)             (None, 10)               

In [47]:
network.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
network.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=128, epochs=10)

network.evaluate(X_test, y_test)

Train on 58605 samples, validate on 14652 samples
Epoch 1/10
58605/58605 [==============================] - 35s 597us/sample - loss: 3.9699 - accuracy: 0.5656 - val_loss: 0.8315 - val_accuracy: 0.7503
Epoch 2/10
58605/58605 [==============================] - 35s 592us/sample - loss: 0.7459 - accuracy: 0.7769 - val_loss: 0.7399 - val_accuracy: 0.7774
Epoch 3/10
58605/58605 [==============================] - 35s 597us/sample - loss: 0.6192 - accuracy: 0.8145 - val_loss: 0.6461 - val_accuracy: 0.8099
Epoch 4/10
58605/58605 [==============================] - 34s 588us/sample - loss: 0.5411 - accuracy: 0.8391 - val_loss: 0.6158 - val_accuracy: 0.8220
Epoch 5/10
58605/58605 [==============================] - 35s 590us/sample - loss: 0.4819 - accuracy: 0.8539 - val_loss: 0.5990 - val_accuracy: 0.8306
Epoch 6/10
58605/58605 [==============================] - 35s 589us/sample - loss: 0.4415 - accuracy: 0.8656 - val_loss: 0.6222 - val_accuracy: 0.8266
Epoch 7/10
58605/58605 [====================

[0.9038421362638474, 0.80646896]